In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
with open('train_64_nparray.atikeep', 'rb') as handle:
    X,y = pickle.load(handle)

In [3]:
#load everything
import torch
from torchvision import datasets, transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

import sys, time
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [7]:
from sklearn import preprocessing

In [8]:
le = preprocessing.LabelEncoder()

In [9]:
le.fit(y)

LabelEncoder()

In [11]:
len(le.classes_)

39

In [12]:
y_lebeled = le.transform(y)

In [13]:
y_lebeled[:5]

array([29, 26,  4, 17, 12])

In [15]:
#please create tensors from these numpy array
#torch.from_numpy (copy)  or torch.tensor  (not a copy!)
inputs  = torch.tensor(X)
targets = torch.tensor(y_lebeled)

#please print the shape of these tensors
#use either .size() or .shape
inputs.shape, targets.shape


(torch.Size([43040, 64, 64, 3]), torch.Size([43040]))

In [16]:
from torch.utils.data import TensorDataset


In [17]:
ds = TensorDataset(inputs, targets)

In [18]:
X.shape

(43040, 64, 64, 3)

In [19]:
43040 - 8600

34440

In [20]:
train_set, val_set = torch.utils.data.random_split(ds, [34440, 8600])